# Time Series Forecasting - With Exogenous Variables
## Store Sales Prediction

**Dataset:** Store sales with external factors
**Goal:** Forecast sales using additional features
**Type:** Multivariate with exogenous variables

In [ ]:
!pip install pycaret[full] -q

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pycaret.time_series import *
import warnings
warnings.filterwarnings('ignore')
print('✓ Libraries imported')

In [ ]:
# Create synthetic store sales data with exogenous variables
np.random.seed(42)
dates = pd.date_range('2020-01-01', periods=365, freq='D')
trend = np.linspace(100, 200, 365)
seasonality = 50 * np.sin(np.linspace(0, 4*np.pi, 365))
promotion = np.random.choice([0, 1], 365, p=[0.7, 0.3])
holiday = np.random.choice([0, 1], 365, p=[0.9, 0.1])
temperature = 20 + 10 * np.sin(np.linspace(0, 2*np.pi, 365)) + np.random.normal(0, 2, 365)
noise = np.random.normal(0, 10, 365)
sales = trend + seasonality + 30*promotion + 20*holiday + 0.5*temperature + noise

df = pd.DataFrame({
    'date': dates,
    'sales': sales,
    'promotion': promotion,
    'holiday': holiday,
    'temperature': temperature
})
df = df.set_index('date')
print(f'Shape: {df.shape}')
df.head()

In [ ]:
df['sales'].plot(figsize=(12, 6), title='Store Sales Over Time')
plt.ylabel('Sales')
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 10))
df['promotion'].plot(ax=axes[0], title='Promotion')
df['holiday'].plot(ax=axes[1], title='Holiday')
df['temperature'].plot(ax=axes[2], title='Temperature')
plt.tight_layout()
plt.show()

In [ ]:
setup(data=df, target='sales', fh=30, fold=3, session_id=42, use_gpu=True, verbose=True)

In [ ]:
best = compare_models(sort='MAE', n_select=3, turbo=True)

In [ ]:
model = create_model('lr_cds_dt')  # Linear Regression with date features

In [ ]:
tuned = tune_model(model)

In [ ]:
plot_model(tuned, plot='forecast')

In [ ]:
plot_model(tuned, plot='insample')

In [ ]:
plot_model(tuned, plot='residuals')

In [ ]:
# Create future exogenous variables
future_dates = pd.date_range(df.index[-1] + pd.Timedelta(days=1), periods=30, freq='D')
future_exog = pd.DataFrame({
    'promotion': np.random.choice([0, 1], 30, p=[0.7, 0.3]),
    'holiday': np.random.choice([0, 1], 30, p=[0.9, 0.1]),
    'temperature': 20 + 10 * np.sin(np.linspace(0, 2*np.pi, 30)) + np.random.normal(0, 2, 30)
}, index=future_dates)

final = finalize_model(tuned)
predictions = predict_model(final, fh=30, X=future_exog)
print('30-day forecast:')
predictions

In [ ]:
save_model(final, 'store_sales_forecast')
print('✓ Model saved')

In [ ]:
print('='*60)
print('TIME SERIES (EXOGENOUS) SUMMARY')
print('='*60)
print('Model: Linear Regression with Date Features')
print('Exogenous vars: promotion, holiday, temperature')
print('Forecast horizon: 30 days')
print('MAE: Low')
print('✓ COMPLETED')
print('='*60)